In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import plotly.graph_objects as go
import cv2
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from PIL import ImageFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
#from keras.preprocessing.image import ImageDataGenerator
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten , Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 20.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.2/902.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114250 sha256=23a32925482f22489a6ae831005754a2793ff03c21473e50774082b1740c769d
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully u

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Conv2D, BatchNormalization, Activation, 
                                     AveragePooling2D, UpSampling2D, concatenate, Dense, Flatten, Dropout)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, precision_recall_fscore_support
from skimage import measure
from sklearn.mixture import GaussianMixture
from sklearn.utils import shuffle

import cv2
import math
import os
import glob
import yolov5  # Assuming YOLOv5 is installed for object detection


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
3
6569 821 822
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


NameError: name 'DSPP' is not defined

In [22]:
def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if not height:
            if hist[i] > 0:
                height.append(hist[i])
                position.append(i)
        else:
            if hist[i] > height[-1]:
                height.append(hist[i])
                position.append(i)
            elif hist[i] < height[-1]:
                while height and height[-1] > hist[i]:
                    maxHeight = height.pop()
                    area = maxHeight * (i - position[-1])
                    if area > maxArea[0]:
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if not height:
                        break
                position.append(last_position)
                height.append(hist[i])
    while height:
        maxHeight = height.pop()
        last_position = position.pop()
        area = maxHeight * (len(hist) - last_position)
        if area > maxArea[0]:
            maxArea = (area, len(hist), last_position)
    return maxArea

def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r - 1]
            addMat[r][img[r] == 0] = 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3] + 1 - maxArea[0] / abs(maxArea[1] - maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])
def cropCircle(img):
    """This function is replaced with YOLO detection for precise bounding box cropping."""
    return yolo_detect_and_crop(img)  # Directly using YOLO integration for precise cropping

def extract_roi(img):
    """Extracts Region of Interest using improved cervix segmentation."""
    cropped_img = cropCircle(img)
    if cropped_img is None:
        return None
    imgLab = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2LAB)
    w, h = cropped_img.shape[:2]
    Ra = np.zeros((w * h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w / 2 - i) ** 2 + (h / 2 - j) ** 2)
            Ra[i * h + j, 0] = R
            Ra[i * h + j, 1] = min(imgLab[i][j][1], 150)
    Ra[:, 0] /= max(Ra[:, 0])
    Ra[:, 0] *= 1.0
    Ra[:, 1] /= max(Ra[:, 1])

    g = GaussianMixture(n_components=2, covariance_type='diag', random_state=0, init_params='kmeans')
    g.fit(shuffle(Ra, random_state=0)[:1000])
    labels = g.predict(Ra)
    labels_2D = np.reshape(labels + 1, (w, h))
    cervix_cluster = np.argmax(np.bincount(labels)) + 1

    mask = (labels_2D == cervix_cluster).astype('uint8')
    return mask * cropped_img


In [8]:
# Implement YOLO stage for bounding box detection
def yolo_detect_and_crop(image_path):
    """Detects cervical regions using YOLO and crops the ROI."""
    yolo_model = yolov5.load('yolov5s')  # Load YOLOv5 model (or replace with custom-trained model)
    results = yolo_model(image_path)
    bbox = results.xyxy[0]  # Extract bounding boxes
    if len(bbox) > 0:
        bbox = bbox[0]  # Select the first bounding box
        x_min, y_min, x_max, y_max = map(int, bbox[:4])
        img = cv2.imread(image_path)
        cropped_img = img[y_min:y_max, x_min:x_max]
        return cropped_img
    return None

In [9]:
# Prepare dataset splits

root_dir = '../input/intel-mobileodt-cervical-cancer-screening'
train_dir = os.path.join(root_dir,'train', 'train')

type1_dir = os.path.join(train_dir, 'Type_1')
type2_dir = os.path.join(train_dir, 'Type_2')
type3_dir = os.path.join(train_dir, 'Type_3')

train_type1_files = glob.glob(type1_dir+'/*.jpg')
train_type2_files = glob.glob(type2_dir+'/*.jpg')
train_type3_files = glob.glob(type3_dir+'/*.jpg')

added_type1_files  =  glob.glob(os.path.join(root_dir, "additional_Type_1_v2", "Type_1")+'/*.jpg')
added_type2_files  =  glob.glob(os.path.join(root_dir, "additional_Type_2_v2", "Type_2")+'/*.jpg')
added_type3_files  =  glob.glob(os.path.join(root_dir, "additional_Type_3_v2", "Type_3")+'/*.jpg')


type1_files = train_type1_files + added_type1_files
type2_files = train_type2_files + added_type2_files
type3_files = train_type3_files + added_type3_files

# create dataframe of file and labels
files = {'filepath': type1_files + type2_files + type3_files,
          'label': ['Type 1']* len(type1_files) + ['Type 2']* len(type2_files) + ['Type 3']* len(type3_files)}

files_df = pd.DataFrame(files).sample(frac=1, random_state= 1).reset_index(drop=True)
# describe the dataframe
files_df.describe()

,filepath,label
count,8215,8215
unique,8215,3
top,../input/intel-mobileodt-cervical-cancer-scree...,Type 2
freq,1,4348


In [10]:
# check for duplicates
len(files_df[files_df.duplicated(subset=['filepath'])])

# check for damaged files
bad_files = []
for path in (files_df['filepath'].values):
    try:
        img = Image.open(path)
    except:
        index = files_df[files_df['filepath']==path].index.values[0]
        bad_files.append(index)
print(len(bad_files))

# drop the damaged files
files_df.drop(bad_files, inplace=True)

# check unique labels
files_df['label'].unique()

# get count of each type 
type_count = pd.DataFrame(files_df['label'].value_counts()).rename(columns= {'label': 'Num_Values'})


3


In [12]:
#  split the data into train  and validation set
train_df, eval_df = train_test_split(files_df, test_size= 0.2, stratify= files_df['label'], random_state= 1)
val_df, test_df = train_test_split(eval_df, test_size= 0.5, stratify= eval_df['label'], random_state= 1)
print(len(train_df), len(val_df), len(test_df))

# loads images from dataframe
def load_images(dataframe):
    features = []
    filepaths = dataframe['filepath'].values
    labels = dataframe['label'].values
    
    for path in filepaths:
        img = cv2.imread(path)
        resized_img = cv2.resize(img, (224, 224))
        features.append(np.array(resized_img))
    return np.array(features), np.array(labels)
# load training and evaluation data
train_features, train_labels = load_images(train_df)
val_features, val_labels = load_images(val_df)
test_features, test_labels = load_images(test_df)

# normalize the features
X_train = train_features/255
X_val  = val_features/255
X_test  = test_features/255


6569 821 822


In [13]:
from tensorflow.keras.utils import to_categorical
# Fit label encoder
le = LabelEncoder().fit(['Type 1', 'Type 2', 'Type 3'])
# Convert labels to integer encoding first, then to one-hot encoding
y_train = to_categorical(le.transform(train_labels), num_classes=3)
y_val = to_categorical(le.transform(val_labels), num_classes=3)
y_test = to_categorical(le.transform(test_labels), num_classes=3)

# initialize image data generator for training and evaluation sets
train_datagen = ImageDataGenerator(
                                rotation_range = 40,
                                zoom_range = 0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                horizontal_flip=True,
                                vertical_flip = True)

eval_datagen = ImageDataGenerator()


In [14]:
BATCH_SIZE= 32
train_gen = train_datagen.flow(X_train, y_train, batch_size= BATCH_SIZE)
val_gen = eval_datagen.flow(X_val, y_val, batch_size= BATCH_SIZE)
test_gen = eval_datagen.flow(X_test, y_test, batch_size= BATCH_SIZE)

In [15]:
# def DSPP(input_tensor, l2_rate=1e-4):
#     pool = AveragePooling2D(pool_size=(input_tensor.shape[1], input_tensor.shape[2]))(input_tensor)
#     pool = Conv2D(128, 1, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(pool)
#     pool = UpSampling2D(size=(input_tensor.shape[1] // pool.shape[1], input_tensor.shape[2] // pool.shape[2]), interpolation="bilinear")(pool)

#     dilated_1 = Conv2D(128, 3, dilation_rate=1, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(input_tensor)
#     dilated_6 = Conv2D(128, 3, dilation_rate=6, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(input_tensor)
#     dilated_12 = Conv2D(128, 3, dilation_rate=12, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(input_tensor)
#     dilated_18 = Conv2D(128, 3, dilation_rate=18, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(input_tensor)

#     x = concatenate([pool, dilated_1, dilated_6, dilated_12, dilated_18])
#     return Conv2D(128, 1, padding="same", activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(x)

In [24]:
# Modified ResNet-50 model without DSPP
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
for layer in base_model.layers[:-10]:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dense(3, activation='softmax')(x)

model1 = Model(inputs=base_model.input, outputs=x)
model1.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

model2 = Model(inputs=base_model.input, outputs=x)
model2.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Assume train_df is a pandas DataFrame with a 'label' column
# Convert labels to a NumPy array (if not already)
labels = train_df['label'].values

# Compute class weights
weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# Convert to a dictionary {class_index: weight}
unique_classes = np.unique(labels)
class_weight_dict = dict(zip(unique_classes, weights))

# Then pass it into model.fit(...)
model1.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(patience=5)
    ],
    class_weight=class_weight_dict
)


Epoch 1/30
206/206 ━━━━━━━━━━━━━━━━━━━━ 792s 4s/step - accuracy: 0.4408 - loss: 3.4077 - val_accuracy: 0.2948 - val_loss: 1.2271 - learning_rate: 0.0010
Epoch 2/30
206/206 ━━━━━━━━━━━━━━━━━━━━ 777s 4s/step - accuracy: 0.4775 - loss: 1.0966 - val_accuracy: 0.5201 - val_loss: 1.0257 - learning_rate: 0.0010
Epoch 3/30
206/206 ━━━━━━━━━━━━━━━━━━━━ 795s 4s/step - accuracy: 0.4890 - loss: 1.0545 - val_accuracy: 0.5286 - val_loss: 1.0210 - learning_rate: 0.0010
Epoch 4/30
 17/206 ━━━━━━━━━━━━━━━━━━━━ 10:53 3s/step - accuracy: 0.5057 - loss: 1.0703

In [20]:
# Training two models with different learning rates and datasets
history1 = model1.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(patience=5)],
    class_weight=class_weight_dict
)

history2 = model2.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(patience=5)],
    class_weight=class_weight_dict
)


AttributeError: 'NumpyArrayIterator' object has no attribute 'classes'

In [ ]:
# Ensembling predictions
def ensemble_predictions(model1, model2, test_gen):
    predictions1 = model1.predict(test_gen)
    predictions2 = model2.predict(test_gen)
    return (predictions1 + predictions2) / 2

ensemble_preds = ensemble_predictions(model1, model2, test_data)
y_true = test_gen.classes
y_pred = np.argmax(ensemble_preds, axis=-1)

# Evaluation
eval_metrics = model1.evaluate(test_data)  # Using model1 metrics as a reference
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
print(f"Test Accuracy: {eval_metrics[1] * 100:.2f}%")
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
